In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
ndf_unfiltered = pd.read_csv('ndf_hmmer_search_hits.csv')

In [3]:
ndf_unfiltered['Subunit'].value_counts()

Subunit
NDUFA9     445798
NDUFA10      4285
NDUFS4       2670
NDUFA12      2513
NDUFS6        919
NDUFV3          6
NDUFC1          2
NDUFB10         1
Name: count, dtype: int64

In [4]:
# Boolean table for Nuo Complex I subunits variation data
nuo_bool = pd.read_csv('nuo_bool_final.csv')

In [5]:
nuo_bool.head()

,GenomeFile,Accession,Superkingdom,Phylum,Class,Order,Family,Genus,Species,Replicon,...,NuoH,NuoI,NuoJ,NuoK,NuoL,NuoM,NuoN,Variation,Cluster,Strand
0,GCA_000009085.1_ASM908v1_genomic.fna,AL111168.1,Bacteria,Campylobacterota,Epsilonproteobacteria,Campylobacterales,Campylobacteraceae,Campylobacter,Campylobacter jejuni,Chromosome,...,True,True,True,True,True,True,True,Nuo-like,NaN,NaN
1,GCF_001457695.1_NCTC11351_genomic.fna,NZ_LN831025.1,Bacteria,Campylobacterota,Epsilonproteobacteria,Campylobacterales,Campylobacteraceae,Campylobacter,Campylobacter jejuni,Chromosome,...,True,True,True,True,True,True,True,Nuo-like,NaN,NaN
2,GCF_037680265.1_ASM3768026v1_genomic.fna,NZ_CP148213.1,Bacteria,Campylobacterota,Epsilonproteobacteria,Campylobacterales,Campylobacteraceae,Campylobacter,Campylobacter jejuni,Chromosome,...,True,True,True,True,True,True,True,Nuo-like,NaN,NaN
3,GCA_033097125.1_ASM3309712v1_genomic.fna,AP028392.1,Bacteria,Campylobacterota,Epsilonproteobacteria,Campylobacterales,Campylobacteraceae,Campylobacter,Campylobacter jejuni,Chromosome,...,True,True,True,True,True,True,True,Nuo-like,NaN,NaN
4,GCF_025232495.1_ASM2523249v1_genomic.fna,NZ_CP012237.1,Bacteria,Campylobacterota,Epsilonproteobacteria,Campylobacterales,Campylobacteraceae,Campylobacter,Campylobacter jejuni,Chromosome,...,True,True,True,True,True,True,True,Nuo-like,NaN,NaN


In [6]:
ndf_unfiltered.sort_values('BitScore', ascending=False).head()

,Accession,Replicon,ProteomeFile,Species,ProteinAccession,evalue,BitScore,Bias,SequenceDesc,Subunit,Start,End,log10evalue,EstProtLength
250898,CP048788.1,Chromosome,GCA_012932215.1_ASM1293221v1_cds_proteins.faa,Roseobacter ponti,CP048788.1_3625,5.600000e-150,497.7,0.1,# 3766663 # 3767646 # -1 # ID=1_3625;partial=0...,NDUFA9,3766663,3767646,-149.251812,328
313207,CP090021.1,Chromosome,GCA_021391315.1_ASM2139131v1_cds_proteins.faa,Cereibacter azotoformans,CP090021.1_2799,3.300000e-148,492.1,0.8,# 2825453 # 2826439 # -1 # ID=1_2799;partial=0...,NDUFA9,2825453,2826439,-147.481486,329
30944,JAGFXX010000001.1,Chromosome,GCA_017599285.1_ASM1759928v1_cds_proteins.faa,Cereibacter azotoformans,JAGFXX010000001.1_2562,2.400000e-148,492.1,0.8,# 2586504 # 2587490 # 1 # ID=1_2562;partial=00...,NDUFA9,2586504,2587490,-147.619789,329
246241,CP089965.1,Chromosome,GCA_022227035.1_ASM2222703v1_cds_proteins.faa,Cereibacter azotoformans,CP089965.1_2823,3.600000e-148,492.1,0.8,# 2843382 # 2844368 # -1 # ID=1_2823;partial=0...,NDUFA9,2843382,2844368,-147.443697,329
183734,NZ_CP033446.1,Chromosome,GCF_003846385.1_ASM384638v1_cds_proteins.faa,Cereibacter sphaeroides,NZ_CP033446.1_261,6.300000e-147,487.8,0.1,# 272601 # 273587 # 1 # ID=1_261;partial=00;st...,NDUFA9,272601,273587,-146.200659,329


In [7]:
colors = ["#a5b1c2", "#f7b731", "#20bf6b", "#45aaf2", "#45aaf2", "#3867d6", "#a55eea", "#0fb9b1", '#4b6584', "#eb3b5a"]
labels = ['Nuo-Partial', 'Nuo13', 'Nuo14', 'Nuo-like', 'Nuo14-EFG', 'Nuo13-EFG', 'Nuo14-EF', 'Nuo12', 'Existing Annotation', 'Absent']

complex_colors = dict(zip(labels, colors))

In [8]:
ndf_unfiltered['Subunit'].value_counts()

Subunit
NDUFA9     445798
NDUFA10      4285
NDUFS4       2670
NDUFA12      2513
NDUFS6        919
NDUFV3          6
NDUFC1          2
NDUFB10         1
Name: count, dtype: int64

In [9]:
# List of subunits to analyze
subunits = sorted(ndf_unfiltered['Subunit'].unique())

# Directory to save plots
output_dir = "acc_kde_plots"
os.makedirs(output_dir, exist_ok=True)

for subunit in subunits:
    # Filter HMMER data for the current subunit
    filtered_hmmer_data = ndf_unfiltered.loc[
        ndf_unfiltered['Subunit'] == subunit, 
        ['Accession', 'log10evalue']
    ].drop_duplicates()

    # Extract unique accession-variation mappings
    unique_variations = nuo_bool[['Accession', 'Variation']].drop_duplicates()

    # Identify missing accessions and assign 'Absent' variation
    missing_accessions = filtered_hmmer_data.loc[
        ~filtered_hmmer_data['Accession'].isin(unique_variations['Accession']), 
        ['Accession']
    ].assign(Variation='Absent')

    # Combine variation data
    variation_status = pd.concat([unique_variations, missing_accessions], ignore_index=True).drop_duplicates()

    # Merge filtered HMMER data with variation status
    plot_data = filtered_hmmer_data.merge(variation_status, on='Accession', how='left')

    # Generate KDE plot only if there is valid data
    if 'Variation' in plot_data.columns and not plot_data.empty:
        plt.figure(figsize=(8, 6))
        sns.kdeplot(data=plot_data, x='log10evalue', hue='Variation', palette=complex_colors)
        plt.title(f"KDE Plot for {subunit}")
        plt.xlabel("log10(E-value)")
        plt.ylabel("Density")

        # Save the plot
        plot_filename = os.path.join(output_dir, f"{subunit}_kde.png")
        plt.savefig(plot_filename, dpi=300)
        plt.close()  # Close the plot to avoid overlap in the loop
        print(f"Saved: {plot_filename}")
    else:
        print(f"Warning: No valid data for {subunit}, skipping plot.")


Saved: acc_kde_plots/NDUFA10_kde.png
Saved: acc_kde_plots/NDUFA12_kde.png
Saved: acc_kde_plots/NDUFA9_kde.png
Saved: acc_kde_plots/NDUFB10_kde.png
Saved: acc_kde_plots/NDUFC1_kde.png
Saved: acc_kde_plots/NDUFS4_kde.png
Saved: acc_kde_plots/NDUFS6_kde.png
Saved: acc_kde_plots/NDUFV3_kde.png


In [10]:
# Based on the KDE distibution, e-value cutoffs are selected

E_VALUE_CUTOFF = {'NDUFS4' : -25, 'NDUFA12' : -35, 'NDUFA9' : -75}

In [11]:
selected_ndf = ndf_unfiltered[ndf_unfiltered['Subunit'].isin(E_VALUE_CUTOFF.keys())]

In [12]:
selected_ndf = selected_ndf[selected_ndf['log10evalue'] <= selected_ndf['Subunit'].map(E_VALUE_CUTOFF).fillna(float('inf'))]

In [13]:
selected_ndf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9643 entries, 110 to 456030
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Accession         9643 non-null   object 
 1   Replicon          9643 non-null   object 
 2   ProteomeFile      9643 non-null   object 
 3   Species           9643 non-null   object 
 4   ProteinAccession  9643 non-null   object 
 5   evalue            9643 non-null   float64
 6   BitScore          9643 non-null   float64
 7   Bias              9643 non-null   float64
 8   SequenceDesc      9643 non-null   object 
 9   Subunit           9643 non-null   object 
 10  Start             9643 non-null   int64  
 11  End               9643 non-null   int64  
 12  log10evalue       9643 non-null   float64
 13  EstProtLength     9643 non-null   int64  
dtypes: float64(4), int64(3), object(7)
memory usage: 1.1+ MB


In [14]:
# Set output directory
proteomes_dir = '/proteomes'

output_dir = '/acc_seqs'

for subunit in sorted(selected_ndf['Subunit'].unique()):
    selected_selected_ndf = selected_ndf[selected_ndf['Subunit'] == subunit]
    
    # File to store the combined sequences
    output_fasta = f"{output_dir}/{subunit}_filtered_hits.fasta"
    
    # Create a mapping of proteome files to protein accessions
    proteome_accessions = selected_selected_ndf.groupby('ProteomeFile')['ProteinAccession'].unique().to_dict()
    
    # Open the output file once
    with open(output_fasta, 'w') as output_handle:
        # Process each proteome file only once
        for proteome_file, accessions in proteome_accessions.items():
            proteome_file_fp = os.path.join(proteomes_dir, proteome_file)
            try:
                # Parse the proteome file and extract sequences for needed accessions
                needed_accessions = set(accessions)
                sequences_found = 0
                
                for record in SeqIO.parse(proteome_file_fp, 'fasta'):
                    if record.id in needed_accessions:
                        # Clean the sequence
                        protein_sequence = str(record.seq).replace('*', '')
                        
                        # Create a SeqRecord with the cleaned sequence
                        seq_record = SeqRecord(
                            seq=protein_sequence,
                            id=record.id,
                            description=record.description
                        )
                        
                        # Write the SeqRecord to the output FASTA file
                        SeqIO.write(seq_record, output_handle, 'fasta')
                        
                        sequences_found += 1
                        needed_accessions.remove(record.id)
                        
                        # Break if all needed accessions have been found
                        if not needed_accessions:
                            break
                if sequences_found == 0:
                    print(f"No sequences found in {proteome_file_fp} for the specified accessions.")
            except FileNotFoundError:
                print(f"File not found: {proteome_file_fp}")
            except Exception as e:
                print(f"An error occurred with file {proteome_file_fp}: {e}")

These sequences are further used to build phylogenetic trees